Comments from [Go by Example](https://gobyexample.com/worker-pools).

# Channels
Channels are the pipes that connect concurrent goroutines. You can send values into channels from one goroutine and receive those values into another goroutine.

In [15]:
import "fmt"

In [19]:
messages := make(chan string) // Create a new unbuffered channel

go func() { messages <- "ping" }() // In separate go-routine, put the value "ping" in our channel (so our code doesn't block, more on this later)

msg := <-messages // Get our value from the channel, and put it in the variable msg
;

In [20]:
msg

ping

## Unbuffered channels

By default channels are unbuffered, meaning that they will only accept sends (chan <-) if there is a corresponding receive (<- chan) ready to receive the sent value.
- A send to a buffered channels block
- A receive to a buffered channels blocks, until we actually receive a value from the channel

In [21]:
import "fmt"

In [22]:
messages := make(chan string)

blockText := "NOT UPDATED" // We will use this to know when the code below stopped blocking

go func() {
    messages <- "hello" // Note how the code just blocks here
    
    blockText = "UPDATED"
}()
;

In [23]:
fmt.Printf("The channel now has an element in it\n")

The channel now has an element in it


37 <nil>

The code above is blocking, because the value that is in the channel has not been read yet

In [24]:
fmt.Printf("We can see that the value of our 'blockText' is:\n%s\n", blockText)

We can see that the value of our 'blockText' is:
NOT UPDATED


61 <nil>

Let's read from the channel to unblock the goroutine's code

In [25]:
msg := <-messages

In [26]:
fmt.Printf("We can see that the value of our 'blockText' is now:\n%s\n", blockText)

We can see that the value of our 'blockText' is now:
UPDATED


61 <nil>

## Buffered channels
Buffered channels accept a limited number of values without a corresponding receiver for those values.
Because this channel is buffered, we can send these values into the channel without a corresponding concurrent receive, until the channel gets full, then it blocks.

In [27]:
import "fmt"

In [28]:
fruitChan := make(chan string, 3)

fruits := []string{"apple", "orange", "grape", "watermelon", "avocado", "grapefruit", "banana"}

nextFruit := ""

go func() {
    for _, f := range fruits {
        nextFruit = f
        fruitChan <- nextFruit
    }
    nextFruit = ""
}()

;

Note how the above goroutine was able to put multiple items in the channel without blocking.
The blocking only happend once the channel was full.

In [29]:
fmt.Printf("The channel is now full with '%d' elements\n", len(fruitChan))
fmt.Printf("There is a '%s' waiting for a slot in the channel\n", nextFruit)

The channel is now full with '3' elements
There is a 'watermelon' waiting for a slot in the channel


58 <nil>

Let's keep track of the fruits we get from the channel in the `basket` slice, and read one element from the channel

In [30]:
basket := []string{}
basket = append(basket, <-fruitChan)
fmt.Printf("Our basket now has: %s\n", basket)
fmt.Printf("There is a '%s' waiting for a slot in the channel\n", nextFruit)

Our basket now has: [apple]
There is a 'watermelon' waiting for a slot in the channel


58 <nil>

Let's read another element from the channel

In [31]:
basket = append(basket, <-fruitChan)
fmt.Printf("Our basket now has: %s\n", basket)
fmt.Printf("There is a '%s' waiting for a slot in the channel\n", nextFruit)

Our basket now has: [apple orange]
There is a 'avocado' waiting for a slot in the channel


55 <nil>

Let's now iterate over each element as they are received from the channel

In [32]:
go func() {
    for f := range fruitChan { // fruitChan emptying loop
        basket = append(basket, f)
    }
    basket = append(basket, "olive") // We will use this condition to know if the code above is blocking or not
}()

In [33]:
fmt.Printf("Our basket now has: %s\n", basket)

Our basket now has: [apple orange grape watermelon avocado grapefruit banana]


78 <nil>

Note that the `fruitChan emptying loop` above is still blocking though, we know it is blocking because the last element in `basket` is not `"olive"`.
If we close the channel the `fruitChan emptying loop`, will no longer block. And the "olive" will be appended to the end of the `basket`

In [34]:
close(fruitChan)
fmt.Printf("Our basket now has: %s\n", basket)

Our basket now has: [apple orange grape watermelon avocado grapefruit banana]


78 <nil>

Note how we can now see `"olive"` at the end of out basket

## Notes (from https://dave.cheney.net/2013/04/30/curious-channels)

#### Writing to a closed channel cauises a panic

#### Reading from a closed channel *never* blocks, if the channel is empty, it returns the zero value of the channel's type

In [35]:
ch := make(chan bool, 2)
ch <- true
ch <- true
close(ch)

for i := 0; i < cap(ch) +1 ; i++ {
        // ok is a boolean reporting whether the communication succeeded.
        //   true: received value was sent on the channel
        //   false: v is zero value because the channel is closed and empty
        v, ok := <- ch 
    fmt.Printf("v: %v, ok: %v\n", v, ok)
}

v: true, ok: true
v: true, ok: true
v: false, ok: false


#### A nil channel always blocks

In [36]:
go func() {
        var ch chan bool
        ch <- true // blocks forever
    
        fmt.Println("never printed")
}()
go func() {
        var ch chan bool
        <- ch // also blocks forever
    
        fmt.Println("also never printed")
}()